В решении задачи будет использоваться модель SASR в сочетании с FMNN (SASR - основная).  
Чтобы спроектировать схему валидации, необходимо ответить на следующие вопросы:  
- Что мы хотим от модели? Ранжирование объектов для пользователя. Т е будем выбирать соответствующие метрики  
- Как будет использоваться модель? Рекомендации будут считаться раз в какой-то период. Т е надо учитывать динамику предпочтений пользователя  
- Какие технические ограничения? Время на обучение, время на построение рекомендаций. Т е на это будем смотреть при проверке работы модели, чтобы были приемлемые затраты  

Определимся с особенностями задачи:
- Рекомендуем для пользователя объекты, с которыми он уже взаимодействовал. Т е в отличие от рекомендаций фильмов и книг не нужно отсеивать объекты, с которыми пользователь уже взаимодейстовал
- Присутствуют пользователи с warm start и cold start. Т е в валидационной и тестовой выборках должны присутствовать пользователи, у которых почти не было взаимодействий.

Существующие подходы для разбиения данных:
- Случайное разбиение
- Хронологическое разбиение
- Разбиение по времени  

В данной задаче больше всего подойдёт разбиение по времени: первые несколько дней - тренировочная выборка, следующие - валидационная, последние - тестовая. Возможен лик данных, т к не все пользователи совершали покупки в эти даты, поэтому для расчёта размера тестовой выборки проведём анализ частоты совершения покупок.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix

In [3]:
df = pd.read_csv('/content/drive/MyDrive/wb_filtered_data.csv')
df.head()

,user_id,item_id,order_ts
0,550614,264,2023-01-01 00:28:09.000000
1,571051,580,2023-01-01 00:41:47.000000
2,571051,180,2023-01-01 00:41:47.000000
3,47164,5135,2023-01-01 00:53:35.000000
4,219072,2668,2023-01-01 01:02:29.000000


Заменим значения id на индексы в списке уникальных элементов

In [4]:
user_index_map = {user:i for i, user in enumerate(df['user_id'].unique())}
item_index_map = {item:i for i, item in enumerate(df['item_id'].unique())}
df['user_id'] = df['user_id'].map(user_index_map)
df['item_id'] = df['item_id'].map(item_index_map)
df.head()

,user_id,item_id,order_ts
0,0,0,2023-01-01 00:28:09.000000
1,1,1,2023-01-01 00:41:47.000000
2,1,2,2023-01-01 00:41:47.000000
3,2,3,2023-01-01 00:53:35.000000
4,3,4,2023-01-01 01:02:29.000000


In [5]:
# Преобразование текста в дату время
df['order_ts'] = pd.to_datetime(df['order_ts'])
# Сортировка данных по времени заказа
df.sort_values('order_ts', inplace=True)
# Выделение даты без времени
df['order_date'] = df['order_ts'].dt.date

In [6]:
df.head()

,user_id,item_id,order_ts,order_date
1692586,284402,3278,2023-01-01 00:00:05,2023-01-01
1940291,463218,2597,2023-01-01 00:00:08,2023-01-01
552,543,120,2023-01-01 00:00:12,2023-01-01
1954320,161063,516,2023-01-01 00:00:13,2023-01-01
652503,43025,1285,2023-01-01 00:00:16,2023-01-01


In [7]:
# Вычисление временного интервала между последовательными покупками для каждого пользователя
df['time_diff'] = df.groupby('user_id')['order_ts'].diff()
# Вычисление средней частоты совершения покупок для каждого пользователя
average_frequency = df.groupby('user_id')['time_diff'].quantile(0.75)
days = average_frequency.mean().days

/usr/local/lib/python3.10/dist-packages/pandas/core/groupby/groupby.py:3212: RuntimeWarning: invalid value encountered in cast
  vals = vals.astype("i8").view(


In [8]:
days

7

In [9]:
df.tail()

,user_id,item_id,order_ts,order_date,time_diff
19196383,41029,431,2023-03-31 23:59:59.409847,2023-03-31,1 days 14:53:01.687394
19236625,205638,256,2023-03-31 23:59:59.475521,2023-03-31,0 days 00:13:38.256089
19233747,603647,1658,2023-03-31 23:59:59.728595,2023-03-31,0 days 00:35:12.471624
19212552,328149,388,2023-03-31 23:59:59.862241,2023-03-31,2 days 15:42:37.854907
17713163,104991,69,2023-03-31 23:59:59.947831,2023-03-31,4 days 03:00:54.333997


In [10]:
start_date = df['order_date'].min()
end_date = df['order_date'].max()
all_days = (end_date - start_date).days + 1
all_days

90

In [11]:
days / all_days

0.07777777777777778

В среднем покупки совершаются пользователями раз в 7 дней, т е это минимальный размер тестовой выборки, но т к это 8% от всех данных (90 дней), можно накинуть пару дней. Валидационную выборку возьмём в 2 раза больше тестовой

In [14]:
# Определение временных промежутков для разделения данных
td_test = all_days - (days + 2)
td_val = td_test - 2 * (days + 2)

In [15]:
print(td_test)
print(td_val)

81
63


In [ ]:
# Разделение данных на обучающую, тестовую и валидационную выборки
train_df = df[(df['order_date'] >= start_date) & (df['order_date'] < start_date + pd.Timedelta(days=td_val))]
valid_df = df[(df['order_date'] >= start_date + pd.Timedelta(days=td_val)) & (df['order_date'] < start_date + pd.Timedelta(days=td_test))]
test_df = df[(df['order_date'] >= start_date + pd.Timedelta(days=td_test)) & (df['order_date'] <= end_date)]

In [ ]:
train_df.head()

,user_id,item_id,order_ts,order_date,time_diff
1692586,284402,3278,2023-01-01 00:00:05,2023-01-01,NaT
1940291,463218,2597,2023-01-01 00:00:08,2023-01-01,NaT
552,543,120,2023-01-01 00:00:12,2023-01-01,NaT
1954320,161063,516,2023-01-01 00:00:13,2023-01-01,NaT
652503,43025,1285,2023-01-01 00:00:16,2023-01-01,NaT


In [ ]:
valid_df.head()

,user_id,item_id,order_ts,order_date,time_diff
14010456,471275,789,2023-03-05 00:00:00.094071,2023-03-05,3 days 09:43:26.422647
13765150,722846,101,2023-03-05 00:00:00.194411,2023-03-05,0 days 00:00:15.036412
14613044,46320,707,2023-03-05 00:00:02.199716,2023-03-05,0 days 00:24:52.186766
13879217,98231,517,2023-03-05 00:00:03.781173,2023-03-05,3 days 03:17:48.671470
14602266,98231,687,2023-03-05 00:00:03.781173,2023-03-05,0 days 00:00:00


In [ ]:
test_df.head()

,user_id,item_id,order_ts,order_date,time_diff
16346206,72055,2013,2023-03-23 00:00:00.044023,2023-03-23,0 days 03:10:58.987287
17553880,95132,98,2023-03-23 00:00:00.719099,2023-03-23,0 days 16:55:08.450543
17299725,95132,11,2023-03-23 00:00:00.719099,2023-03-23,0 days 00:00:00
15871023,95132,1359,2023-03-23 00:00:00.719099,2023-03-23,0 days 00:00:00
17830368,95132,1040,2023-03-23 00:00:00.719099,2023-03-23,0 days 00:00:00


In [ ]:
print(train_df.shape[0]/len(df))
print(valid_df.shape[0]/len(df))
print(test_df.shape[0]/len(df))

0.6835490257403499
0.2119842424918242
0.10446673176782591


In [ ]:
train_df.to_csv('/content/drive/MyDrive/train_df.csv', index=False)
valid_df.to_csv('/content/drive/MyDrive/val_df.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/test_df.csv', index=False)